In [5]:
import requests
import json
import pandas as pd

# Your API Secret Key (replace 'YOUR_SECRET_KEY' with the actual key)
api_key = 'p3a88u9bwh8d8npn53g7htra8cxgeh8y'

# Set the base URL for the API request
base_url = "https://api.healthdata.org/sdg/v1/GetResultsByLocation"

location_id = 149 

# Set the year you are interested in
year = 2023  # Replace with the desired year

# Define the headers with your API key
headers = {
    'Content-Type': 'application/json',
    'Authorization': api_key
}

# Construct the full API request URL
url = f"{base_url}?year={year}&location_id={location_id}"

In [6]:
# Make the GET request to the API
response = requests.get(url, headers=headers)


In [7]:
# Check if the request was successful
if response.status_code == 200:
    # If successful, parse the JSON data
    data = response.json()

    # save the json in a json file
    with open('palestine_.json', 'w') as f:
        json.dump(data, f)

    df = pd.DataFrame(data["results"])
    with open('palestine.csv', 'w') as f:
        df.to_csv(f)

    # Assuming the API returns a URL to download the CSV file
    csv_url = data.get("csv_url")

    if csv_url:
        # Download the CSV file
        csv_response = requests.get(csv_url)

        # Save the CSV file locally
        with open("nigeria_most_deaths.csv", "wb") as file:
            file.write(csv_response.content)

        print("CSV file downloaded and saved as 'nigeria_most_deaths.csv'.")
    else:
        print("CSV URL not found in the response.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    print(f"Response: {response.text}")

CSV URL not found in the response.


In [8]:
print(df.head(1))

   location_id location_name  year_id  indicator_id      indicator_name  \
0          149     Palestine     2023          1000  HIV incidence rate   

   age_group_id age_group_name  sex_id sex_label  scenario scenario_label  \
0            22       All Ages       1     Males        -1          Worse   

  indicator_short                              indicator_description  \
0       HIV Incid  Age-standardized rate of new HIV infections (p...   

        mean_estimate  
0  0.0061116807138815  


In [9]:
import pandas as pd

# Assuming 'df' is your DataFrame obtained from the JSON response

# Step 1: Filter the DataFrame for 'All Ages' category
all_ages_df = df[df['age_group_name'] == 'All Ages']

# Step 2: Select relevant columns and sort by 'mean_estimate'
# Convert 'mean_estimate' to numeric (float) if it's not already
all_ages_df['mean_estimate'] = pd.to_numeric(all_ages_df['mean_estimate'], errors='coerce')

# Group by 'indicator_name' if you want to aggregate (optional)
# Here, we will just take the maximum estimate for each disease
ranked_df = all_ages_df.groupby('indicator_name').agg({'mean_estimate': 'max'}).reset_index()

# Step 3: Sort the DataFrame by 'mean_estimate' in descending order
ranked_df = ranked_df.sort_values(by='mean_estimate', ascending=False).reset_index(drop=True)

# Display the top N diseases (e.g., top 10)
top_diseases = ranked_df.head(10)
print(top_diseases)

                                      indicator_name  mean_estimate
0       Prevalence of 15 neglected tropical diseases    8586.888830
1                    Coverage of measles (two doses)      97.691689
2                                   Coverage of DPT3      93.965210
3                 Coverage of pneumococcal conjugate      93.885276
4  Risk-weighted prevalence of populations using ...      72.470723
5  Coverage of essential health services, as defi...      65.244196
6                        Tuberculosis incidence rate       7.796083
7                                 HIV incidence rate       0.006112
8                             Malaria incidence rate       0.000000


/tmp/ipykernel_82029/2988281542.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ages_df['mean_estimate'] = pd.to_numeric(all_ages_df['mean_estimate'], errors='coerce')


In [10]:
# Assuming 'df' is your DataFrame

# Print all unique values for each specified column
print("Unique values for 'indicator_name':")
print(df['indicator_name'].unique())

print("\nUnique values for 'age_group_name':")
print(df['age_group_name'].unique())

print("\nUnique values for 'sex_label':")
print(df['sex_label'].unique())

print("\nUnique values for 'scenario_label':")
print(df['scenario_label'].unique())

print("\nUnique values for 'indicator_short':")
print(df['indicator_short'].unique())


Unique values for 'indicator_name':
['HIV incidence rate' 'Tuberculosis incidence rate'
 'Malaria incidence rate' 'Prevalence of 15 neglected tropical diseases'
 'Met need for family planning with modern contraception methods'
 'Coverage of essential health services, as defined by the UHC service coverage index'
 'Under-5 mortality rate' 'Neonatal mortality rate'
 'Prevalence of child stunting'
 'Risk-weighted prevalence of populations using unsafe sanitation'
 'Maternal mortality ratio (WHO)' 'Coverage of DPT3'
 'Coverage of measles (two doses)' 'Coverage of pneumococcal conjugate'
 'Prevalence of daily smoking']

Unique values for 'age_group_name':
['All Ages' '15-49 years' 'Under 5' 'Neonatal' 'Age-standardized']

Unique values for 'sex_label':
['Males' 'Females' 'Both sexes']

Unique values for 'scenario_label':
['Worse' 'Reference' 'Better']

Unique values for 'indicator_short':
['HIV Incid' 'TB Incid' 'Malaria Incid' 'NTD Prev' 'FP Need Met, Mod'
 'UHC Index' 'Under-5 Mort' 'Neon